## Semantic similarity stuff needs spaCy and BERT. Pulling from one CSV file with mulitple rows
to do:
I need to figure out the preprocessing (I need sentences, leave stop words for now, but maybe get rid of non-alphanumeric)

# Step 1: Set Up the Environment

Install in virtual environment
1. conda create -n bertopic-env python=3.10
2. conda activate bertopic-env
3. conda install -c conda-forge spacy
4. conda install -c conda-forge cupy
5. conda install -c conda-forge bertopic
6. conda install -c conda-forge openpyxl
7. conda install -c conda-forge ipywidgets
8. conda install -c conda-forge jupyter_core
9. python -m spacy download en_core_web_trf


In [ ]:
%pip install spacy
%pip install cupy
%pip install bertopic
%pip install openpyxl
%pip install nltk
%pip install llama_index
%pip install sentence_transformers

In [ ]:
%pip install bertopic

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch

print("CUDA Available: ", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Device Name: ", torch.cuda.get_device_name(0))



# Check for GPU with CUDA core

# Step 1 Load and Preprocess the data with spaCy

In [24]:
# Define the path to your directory containing CSV files
path_to_files = "data/EMPOWER/Interviews/interview_transcripts" 

## New code starts from here

In [ ]:
import pandas as pd
import os
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm

# Ensure necessary nltk resources are downloaded
nltk.download('punkt_tab')

def load_texts_from_csv_directory(directory_path):
    texts = []
    files_processed = 0
    file_names = []
    for filename in tqdm(os.listdir(directory_path)):
        if filename.endswith(".csv"):
            files_processed += 1
            file_names.append(filename)
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)
            if 'text' in df.columns:
                texts.extend(df['text'].apply(lambda text: text.replace('\n', ' ').replace('yeah', '').replace('Yeah', '')).tolist())
            else:
                print(f"Column 'text' not found in {filename}")
    print(f"Total files processed: {files_processed}")
    print("Files processed:", file_names)
    return texts


# Load texts
documents = load_texts_from_csv_directory(path_to_files)

# Assuming each document is already a string
# Split each document into sentences
sentences = [sent_tokenize(doc) for doc in documents]
sentences = [sentence for doc in sentences for sentence in doc]

print("Processed Sentences:", sentences)


In [ ]:
print(documents[0:5])

In [ ]:
print(f"Number of sentences: {len(sentences)}")
print(sentences[0:20])

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(sentences, show_progress_bar=True)

In [7]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired

topic_model = BERTopic(
  embedding_model=embedding_model,
  representation_model=KeyBERTInspired(),
  ctfidf_model=ClassTfidfTransformer(reduce_frequent_words=True),
  top_n_words=25
)
topics, probs = topic_model.fit_transform(sentences, embeddings)

## Topic Reduction

In [ ]:
model = BERTopic(nr_topics=20)

## Merge Selected Topics

In [ ]:
topics_to_merge = [1, 2]
topic_model.merge_topics(sentences, topics, topics_to_merge)

## Make Predictions?

In [ ]:
topics, probs = topic_model.transform(sentences)

## List Top Topics

In [ ]:
topic_model.get_topic_freq().head(11)

## Get Topic info

In [ ]:
topic_model.get_topic_info()

## Most frequent words for a specific topic

In [ ]:
topic_model.get_topic(2)

## Get Document Info

In [ ]:
topic_model.get_document_info(sentences)

## Word cloud of specific topic

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def create_wordcloud(model, topic):
    text = {word: value for word, value in model.get_topic(topic)}
    wc = WordCloud(background_color="white", max_words=1000)
    wc.generate_from_frequencies(text)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.show()
create_wordcloud(topic_model, topic=2)

## Visualize documents

In [ ]:
topic_model.visualize_documents(sentences)

## Intertopic Distance Map

In [ ]:
topic_model.visualize_topics(custom_labels=True)

## Visualize Barchart

In [ ]:
topic_model.visualize_barchart()

## Topic Similarity Matrix Visualization

In [ ]:
topic_model.visualize_heatmap()

## Searching for topics that closely relate to a search term

## List most mentioned named entities in corpus

In [ ]:
import spacy
from collections import Counter

# Load the spaCy model
nlp = spacy.load("en_core_web_trf")  # You can use a larger model for more accuracy


# Function to extract entities
def extract_entities(docs):
    entities = []
    for doc in docs:
        processed_doc = nlp(doc)
        entities.extend([ent.text for ent in processed_doc.ents])
    return entities

# Extract entities from documents
entities = extract_entities(documents)

# Count entities and display most common
entity_counts = Counter(entities)
most_common_entities = entity_counts.most_common()  # You can specify a number inside most_common(n)

# Print the most common entities
for entity, count in most_common_entities:
    print(f"{entity}: {count}")


In [ ]:
similar_topics, similarity = topic_model.find_topics("equity", top_n=5)
topic_model.get_topic(similar_topics[0])

## Semantic SImilarity

In [20]:
import spacy
nlp = spacy.load("en_core_web_trf")

# Define the key document with spacy
key_document = "agency"
key = nlp(key_document)

# Assuming 'sentences' is a list of sentence strings
# We will convert each sentence to a spacy Doc and then compute similarity
for sentence_text in sentences:
    # Process the sentence text with spacy
    sentence_doc = nlp(sentence_text)
    # Compute and print similarity
    similarity = sentence_doc.similarity(key)
    # Print only if similarity is greater than a specific threshold
    if similarity > 0.50:
      print(f"Similarity between '{sentence_text}' and {key_document}: {similarity}")


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example corpus and target sentences
corpus = sentences

# Target sentences whose similar sentences you want to find in the corpus
target_sentences = ["Where students drive the science.",
                    "When its technically student led.",
                    "And it's a continuous process of student agency.",
                    "I noticed that all these projects were student creation, not consumption, which I think is phenomenal.",
                    "Because a lot of these are giving the kids the ability to express themselves and show something about themselves and their vision.",
                    "I put the more ownership a student has the harder they tend to work.",
                    "And they're not just trying to get to the end of it they're actually enjoying, enjoying the process."
                    ]

# Compute embeddings for the corpus and the target sentences
corpus_embeddings = model.encode(corpus)
target_embeddings = model.encode(target_sentences)

# Set a similarity threshold
similarity_threshold = 0.5

# Find and print similar sentences along with context
for target, target_embedding in zip(target_sentences, target_embeddings):
    # Compute cosine similarities
    cos_sim = util.cos_sim(target_embedding, corpus_embeddings)[0]

    # Collect all relevant results above the threshold
    results = []
    for idx, similarity in enumerate(cos_sim):
        if similarity > similarity_threshold:
            results.append((similarity, idx))

    # Sort results by similarity score in descending order
    results.sort(key=lambda x: x[0], reverse=True)

    # Print results
    for similarity, idx in results:
        print(f"Target Sentence: '{target}'")
        print(f"Similar Sentence: '{corpus[idx]}' (Similarity: {similarity:.2f})")
        # Print context: 3 sentences before and after
        context_start = max(0, idx - 3)
        context_end = min(len(corpus), idx + 4)
        context = corpus[context_start:context_end]
        print("Context:")
        for context_sentence in context:
            print(context_sentence)
        print("\n---\n")

    # Print total count and maximum similarity after processing all similarities for a target
    print(f"Total similar sentences found: {len(results)}")
    if results:
        print(f"Highest similarity score: {results[0][0]:.2f}\n\n")


## Save BERTopic Model

In [ ]:
topic_model = BERTopic().fit(sentences)

# Method 1 - safetensors
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("data/outputFiles/output_models/GSQmodel_dir", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

# # Method 2 - pytorch
# embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
# topic_model.save("path/to/my/model_dir", serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)

# # Method 3 - pickle
# topic_model.save("my_model", serialization="pickle")

In [ ]:
import openai
from bertopic.representation import OpenAI

# Fine-tune topic representations with GPT
client = openai.OpenAI(api_key="sk-...")
representation_model = OpenAI(client, model="gpt-3.5-turbo", chat=True)
topic_model = BERTopic(representation_model=representation_model)

## Load BERTopic Model

In [ ]:
# Load from directory
loaded_model = BERTopic.load("path/to/my/model_dir")

# # Load from file
# loaded_model = BERTopic.load("my_model")

# # Load from HuggingFace
# loaded_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

## Extracting Phrases

### Verb phrases

In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd
import os

# Load the spaCy model
nlp = spacy.load("en_core_web_trf")
matcher = Matcher(nlp.vocab)

# Define the directory containing the CSV files
directory_path = '/content/drive/MyDrive/ColabNotebooks/BERTopic/data/EMPOWER/Interviews/interview_transcripts'

# Define the pattern for verb phrases
pattern = [
    {"POS": "VERB"},
    {"POS": "ADV", "OP": "?"},
    {"POS": "PART", "OP": "?"},
    {"POS": "VERB", "OP": "+"}
]

# Add the pattern to the matcher
matcher.add("VERB_PHRASE", [pattern])

# Variable to track the total count of verb phrases across all files
total_verb_phrases_count = 0

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

        # Assume the column containing text is named 'text'
        documents = df['text'].tolist()

        # Process each document to find verb phrases
        for text in documents:
            doc = nlp(text)
            matches = matcher(doc)
            verb_phrases = [doc[start:end].text for match_id, start, end in matches]

            # Update the total count
            total_verb_phrases_count += len(verb_phrases)

            # Print each verb phrase found
            for phrase in verb_phrases:
                print(phrase)

            # Print the total count of verb phrases for each text
            # print(f"Number of verb phrases in '{filename}': {len(verb_phrases)}\n")

# Print the cumulative total count of all verb phrases found
print(f"Total verb phrases count across all documents: {total_verb_phrases_count}")



### Noun Phrases

In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd
import os

# Load the spaCy model
nlp = spacy.load("en_core_web_trf")
matcher = Matcher(nlp.vocab)

# Define the directory containing the CSV files
directory_path = '/content/drive/MyDrive/ColabNotebooks/BERTopic/data/EMPOWER/Interviews/interview_transcripts'

# Define the pattern for noun phrases
pattern = [
    {"POS": "DET", "OP": "?"},  # Optional determiner
    {"POS": "ADJ", "OP": "*"},  # Any number of adjectives
    {"POS": "NOUN", "OP": "+"}  # One or more nouns
]

# Add the pattern to the matcher
matcher.add("NOUN_PHRASE", [pattern])

# Variable to track the total count of noun phrases across all files
total_noun_phrases_count = 0

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

        # Assume the column containing text is named 'text'
        documents = df['text'].tolist()

        # Process each document to find noun phrases
        for text in documents:
            doc = nlp(text)
            matches = matcher(doc)
            noun_phrases = [doc[start:end].text for match_id, start, end in matches]

            # Update the total count
            total_noun_phrases_count += len(noun_phrases)

            # Print each noun phrase found
            for phrase in noun_phrases:
                print(phrase)

            # Print the total count of noun phrases for each text
            # print(f"Number of noun phrases in '{filename}': {len(noun_phrases)}\n")

# Print the cumulative total count of all noun phrases found
print(f"Total noun phrases count across all documents: {total_noun_phrases_count}")


### Verb-object pairs

In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd
import os

# Load the spaCy model
nlp = spacy.load("en_core_web_trf")
matcher = Matcher(nlp.vocab)

# Define the directory containing the CSV files
directory_path = '/content/drive/MyDrive/ColabNotebooks/BERTopic/data/EMPOWER/Interviews/interview_transcripts'

# Define the pattern for verb-object pairs
pattern = [
    {"POS": "VERB"},             # Verb
    {"POS": "DET", "OP": "?"},   # Optional determiner
    {"POS": "ADJ", "OP": "*"},   # Any number of adjectives
    {"POS": "NOUN", "OP": "+"}   # One or more nouns (object)
]

# Add the pattern to the matcher
matcher.add("VERB_OBJECT_PAIR", [pattern])

# Variable to track the total count of verb-object pairs across all files
total_verb_object_pairs_count = 0

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

        # Assume the column containing text is named 'text'
        documents = df['text'].tolist()

        # Process each document to find verb-object pairs
        for text in documents:
            doc = nlp(text)
            matches = matcher(doc)
            verb_object_pairs = [doc[start:end].text for match_id, start, end in matches]

            # Update the total count
            total_verb_object_pairs_count += len(verb_object_pairs)

            # Print each verb-object pair found
            for pair in verb_object_pairs:
                print(pair)

            # Print the total count of verb-object pairs for each text
            # print(f"Number of verb-object pairs in '{filename}': {len(verb_object_pairs)}\n")

# Print the cumulative total count of all verb-object pairs found
print(f"Total verb-object pairs count across all documents: {total_verb_object_pairs_count}")


Subject-Verb-Object

In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd

# Load the spaCy model
nlp = spacy.load("en_core_web_trf")
matcher = Matcher(nlp.vocab)

# Define the directory containing the CSV files
directory_path = '/content/drive/MyDrive/ColabNotebooks/BERTopic/data/EMPOWER/Interviews/interview_transcripts'

# Define the pattern for subject-verb-object triples
pattern = [
    {"POS": "NOUN", "OP": "+"},  # Subject(s)
    {"POS": "VERB"},            # Verb
    {"POS": "DET", "OP": "?"},  # Optional determiner
    {"POS": "ADJ", "OP": "*"},  # Any number of adjectives
    {"POS": "NOUN", "OP": "+"}  # Object(s)
]

# Add the pattern to the matcher
matcher.add("SUBJECT_VERB_OBJECT", [pattern])

# Variable to track the total count of SVO phrases across all files
total_svo_count = 0

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

        # Assume the column containing text is named 'text'
        documents = df['text'].tolist()

        # Process each document to find SVO phrases
        for text in documents:
            doc = nlp(text)
            matches = matcher(doc)
            svo_phrases = [doc[start:end].text for match_id, start, end in matches]

            # Update the total count
            total_svo_count += len(svo_phrases)

            # Print each SVO found
            for svo in svo_phrases:
                print(svo)

            # Print the total count of SVO phrases for each text
            # print(f"Number of SVO phrases in '{filename}': {len(svo_phrases)}\n")

# Print the cumulative total count of all SVO phrases found
print(f"Total SVO phrases count across all documents: {total_svo_count}")

### Adjective-noun pairs

In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd

# Load the spaCy model
nlp = spacy.load("en_core_web_trf")
matcher = Matcher(nlp.vocab)

# Define the directory containing the CSV files
directory_path = '/content/drive/MyDrive/ColabNotebooks/BERTopic/data/EMPOWER/Interviews/interview_transcripts'

# Define the pattern for adjective-noun pairs
pattern = [
    {"POS": "ADJ"},       # Adjective
    {"POS": "NOUN", "OP": "+"}  # One or more nouns
]

# Add the pattern to the matcher
matcher.add("ADJECTIVE_NOUN_PAIR", [pattern])

# Variable to track the total count of adjective-noun pairs across all files
total_adjective_noun_pairs_count = 0

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

        # Assume the column containing text is named 'text'
        documents = df['text'].tolist()

        # Process each document to find adjective-noun pairs
        for text in documents:
            doc = nlp(text)
            matches = matcher(doc)
            adjective_noun_pairs = [doc[start:end].text for match_id, start, end in matches]

            # Update the total count
            total_adjective_noun_pairs_count += len(adjective_noun_pairs)

            # Print each adjective-noun pair found
            for pair in adjective_noun_pairs:
                print(pair)

            # Print the total count of adjective-noun pairs for each text
            # print(f"Number of adjective-noun pairs in '{filename}': {len(adjective_noun_pairs)}\n")

# Print the cumulative total count of all adjective-noun pairs found
print(f"Total adjective-noun pairs count across all documents: {total_adjective_noun_pairs_count}")


### I Statements

In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd

# Load the spaCy model
nlp = spacy.load("en_core_web_trf")
matcher = Matcher(nlp.vocab)


# Define the directory containing the CSV files
directory_path = '/content/drive/MyDrive/ColabNotebooks/BERTopic/data/EMPOWER/Interviews/interview_transcripts'

# Define the pattern for "I statement" phrases
pattern = [
    {"TEXT": "I"},      # The pronoun 'I'
    {"POS": "VERB"},    # Followed by a verb
    {"POS": "DET", "OP": "?"},  # Optionally followed by a determiner
    {"POS": "NOUN"}     # And a noun
]

# Add the pattern to the matcher
matcher.add("I_PHRASE", [pattern])

# Variable to track the total count of "I statement" phrases across all files
total_i_phrases_count = 0

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

        # Assume the column containing text is named 'text'
        documents = df['text'].tolist()

        # Process each document to find "I statement" phrases
        for text in documents:
            doc = nlp(text)
            matches = matcher(doc)
            I_PHRASES = [doc[start:end].text for match_id, start, end in matches]

            # Update the total count
            total_i_phrases_count += len(I_PHRASES)

            # Print each "I statement" phrase found
            for phrase in I_PHRASES:
                print(phrase)

            # Print the total count of "I statement" phrases for each text
            # print(f"Number of 'I statement' phrases in '{filename}': {len(I_PHRASES)}\n")

# Print the cumulative total count of all "I statement" phrases found
print(f"Total 'I statement' phrases count across all documents: {total_i_phrases_count}")



### They Statements

In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd

# Load the spaCy model
nlp = spacy.load("en_core_web_trf")
matcher = Matcher(nlp.vocab)

# Define the directory containing the CSV files
directory_path = '/content/drive/MyDrive/ColabNotebooks/BERTopic/data/EMPOWER/Interviews/interview_transcripts'

# Define the pattern for "they statement" phrases
pattern = [
    {"TEXT": "they"},   # The pronoun 'they'
    {"POS": "VERB"},    # Followed by a verb
    {"POS": "DET", "OP": "?"},  # Optionally followed by a determiner
    {"POS": "NOUN"}     # And a noun
]

# Add the pattern to the matcher
matcher.add("THEY_PHRASE", [pattern])

# Variable to track the total count of "they statement" phrases across all files
total_they_phrases_count = 0

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

        # Assume the column containing text is named 'text'
        documents = df['text'].tolist()

        # Process each document to find "they statement" phrases
        for text in documents:
            doc = nlp(text)
            matches = matcher(doc)
            THEY_PHRASES = [doc[start:end].text for match_id, start, end in matches]

            # Update the total count
            total_they_phrases_count += len(THEY_PHRASES)

            # Print each "they statement" phrase found
            for phrase in THEY_PHRASES:
                print(phrase)

            # Print the total count of "they statement" phrases for each text
            # print(f"Number of 'they statement' phrases in '{filename}': {len(THEY_PHRASES)}\n")

# Print the cumulative total count of all "they statement" phrases found
print(f"Total 'they statement' phrases count across all documents: {total_they_phrases_count}")


### They with negation

In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd

# Load the spaCy model
nlp = spacy.load("en_core_web_trf")
matcher = Matcher(nlp.vocab)


# Define the directory containing the CSV files
directory_path = '/content/drive/MyDrive/ColabNotebooks/BERTopic/data/EMPOWER/Interviews/interview_transcripts'

# Define the pattern for "they statement" phrases with negation
pattern = [
    {"LOWER": "they"},  # The pronoun 'they'
    {"POS": "AUX", "OP": "*"},  # Optional auxiliary (might not be directly next to 'they')
    {"DEP": "neg", "OP": "?"},  # Optional negation modifier
    {"POS": "VERB"},  # Verb following 'they' or an auxiliary/negation
    {"POS": "DET", "OP": "?"},  # Optionally followed by a determiner
    {"POS": "NOUN"}  # And a noun
]

# Add the pattern to the matcher
matcher.add("THEY_NEGATION_PATTERN", [pattern])

# Variable to track the total count of "they" phrases with negation across all files
total_they_negation_phrases_count = 0

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

        # Assume the column containing text is named 'text'
        documents = df['text'].tolist()

        # Process each document to find "they" phrases with negation
        for text in documents:
            doc = nlp(text)
            matches = matcher(doc)
            they_negation_phrases = [doc[start:end].text for match_id, start, end in matches]

            # Update the total count
            total_they_negation_phrases_count += len(they_negation_phrases)

            # Print each "they" phrase with negation found
            for phrase in they_negation_phrases:
                print(phrase)

            # Print the total count of "they" phrases with negation for each text
            # print(f"Number of 'they' phrases with negation in '{filename}': {len(they_negation_phrases)}\n")

# Print the cumulative total count of all "they" phrases with negation found
print(f"Total 'they' phrases with negation count across all documents: {total_they_negation_phrases_count}")



### We Statements

In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd
import os

# Load the spaCy model
nlp = spacy.load("en_core_web_trf")
matcher = Matcher(nlp.vocab)

# Define the directory containing the CSV files
directory_path = '/content/drive/MyDrive/ColabNotebooks/BERTopic/data/EMPOWER/Interviews/interview_transcripts'

# Define the pattern for "we statement" phrases
pattern = [
    {"TEXT": "we"},   # The pronoun 'we'
    {"POS": "VERB"},  # Followed by a verb
    {"POS": "DET", "OP": "?"},  # Optionally followed by a determiner
    {"POS": "NOUN"}   # And a noun
]

# Add the pattern to the matcher
matcher.add("WE_PHRASE", [pattern])

# Variable to track the total count of "we statement" phrases across all files
total_we_phrases_count = 0

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

        # Assume the column containing text is named 'text'
        documents = df['text'].tolist()

        # Process each document to find "we statement" phrases
        for text in documents:
            doc = nlp(text)
            matches = matcher(doc)
            WE_PHRASES = [doc[start:end].text for match_id, start, end in matches]

            # Update the total count
            total_we_phrases_count += len(WE_PHRASES)

            # Print each "we statement" phrase found
            for phrase in WE_PHRASES:
                print(phrase)

            # Print the total count of "we statement" phrases for each text
            # print(f"Number of 'we statement' phrases in '{filename}': {len(WE_PHRASES)}\n")

# Print the cumulative total count of all "we statement" phrases found
print(f"Total 'we statement' phrases count across all documents: {total_we_phrases_count}")



# Old but useful code

In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm, trange

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

embeddings = model.encode(sentences)
for embedding in embeddings:
  print(embedding)

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

#encode the sentences
embeddings = model.encode(sentences, convert_to_tensor=True)
#compute the similarity scores
cosine_scores = util.cos_sim(embeddings, embeddings)
#compute/find the highest similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i]
                                                             [j]})
#sort the scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)
for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i],
                                  sentences[j], pair['score']))

In [ ]:
from sentence_transformers import SentenceTransformer

# Load https://huggingface.co/sentence-transformers/all-mpnet-base-v2
model = SentenceTransformer("all-mpnet-base-v2")
embeddings = model.encode(texts)
similarities = model.similarity(embeddings, embeddings)

In [ ]:
print(similarities)

## Sentence Query from folder of csv files

# Multiple queries

In [ ]:
import os
import pandas as pd
from datetime import datetime
from sentence_transformers import SentenceTransformer, util

# Initialize the model
model_name = "multi-qa-mpnet-base-cos-v1"
model = SentenceTransformer(model_name)

# Define your queries
queries = ["How did you modify your science lessons?", "Why did you make changes to your science lesson?"]
query_embeddings = model.encode(queries)

# Specify the folder containing your CSV files
folder_path = '/content/drive/MyDrive/ColabNotebooks/BERTopic/data/EMPOWER/Interviews/interview_transcripts'

# Prepare a list to store results
results = []

# Process each CSV file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # Load the CSV file
        df = pd.read_csv(os.path.join(folder_path, filename))

        # Check if 'text' column exists
        if 'text' in df.columns:
            # Compute embeddings for all passages in the 'text' column
            passage_embeddings = model.encode(df['text'].tolist())

            # Compute similarities for each query
            for query, query_embedding in zip(queries, query_embeddings):
                similarities = util.cos_sim(query_embedding, passage_embeddings)[0]

                # Filter and gather sentences with a similarity score > 0.40
                for index, (text, similarity) in enumerate(zip(df['text'], similarities)):
                    if similarity > 0.40:
                        results.append({
                            'sentence_number': index,
                            'speaker': df['speaker'][index] if 'speaker' in df.columns else 'Unknown',
                            'filename': filename,
                            'query': query,
                            'text': text,
                            'score': float(similarity)
                        })

# Create DataFrame from results
results_df = pd.DataFrame(results)

# Generate the filename based on the date and model used
date_str = datetime.now().strftime("%Y-%m-%d")
output_filename = f'/content/drive/MyDrive/ColabNotebooks/BERTopic/data/EMPOWER/Interviews/interview_transcripts/{date_str}_{model_name}_results.csv'

# Save the results to a CSV file
results_df.to_csv(output_filename, index=False)

print(f"Results saved to {output_filename}")


https://www.youtube.com/watch?v=6yPWtdgs5Sg